## test cells?


In [ ]:
const Mocha = require('mocha');

// call test runner
let mocha;
if (typeof mocha === 'undefined') {
    mocha = new Mocha({
        ui: 'bdd',
        reporter: 'list',
        reporterOptions: {
            stream: false,
        },
        //parallel: true,
        isWorker: true,
        jobs: 10,
        timeout: 10000,
    });
}

function resetTests(suite) {
    suite.tests.forEach(function (t) {
        delete t.state;
        t.timedOut = false;
    });
    suite.suites.forEach(resetTests);
    suite.tests = [];
}

function testCells(cells, ctx = {}) {
    resetTests(mocha.suite);
    if (!cells) {
        cells = 'mocha test the test cell runner';
    }
    if (!cells[0].code) {
        cells = typeof cells === 'string'
            && (!cells.includes('.ipynb') || cells.includes('['))
            ? [importer.interpret(cells)]
            : importer.interpret(cells);
    }
    cells.forEach(r => {
        mocha.suite.emit('pre-require', ctx, r.id, mocha);
        //Object.assign(ctx, Mocha);
        // TODO: add other require statements below to interent includes?
        try {
            console.log('testing cell:', r.id)
            var required = importer.import(r.id, ctx);
            Object.assign(ctx, required);
        } catch (e) {
            console.log(e);
        }
        mocha.suite.emit('require', required, r.id, mocha);
        mocha.suite.emit('post-require', ctx, r.id, mocha);
    });
    return new Promise(resolve => mocha.run(function (failures) {
        resolve(failures);
    }));
}

module.exports = testCells;



mocha test the test cell runner?


In [ ]:
const assert = require('assert');
const sinon = require('sinon');
const proxyquire = require('proxyquire');
const EventEmitter = require('events');
const Mocha = require('mocha')

describe('testCells', function () {
  let testCells;
  let importerStub;
  let mochaRunStub;
  let suiteMock;
  let mochaMock;

  beforeEach(() => {
    suiteMock = new EventEmitter();
    suiteMock.tests = [];
    suiteMock.suites = [];
    suiteMock.emit = sinon.spy();

    mochaRunStub = sinon.stub().callsFake(cb => cb(0));

    mochaMock = function () {
      return {
        suite: suiteMock,
        run: mochaRunStub
      };
    };
    mochaMock.prototype = function () {};
    mochaMock.prototype.ui = sinon.stub();
    mochaMock.prototype.reporter = sinon.stub();
    mochaMock.prototype.timeout = sinon.stub();

    importerStub = {
      interpret: sinon.stub(),
      import: sinon.stub()
    };

    testCells = importer.import('test cells', {
      require: function(string) {
        return proxyquire(string, {
          'mocha': mochaMock,
        })
      },
      importer: importerStub,
      cached: false,
     });
  });

  it('should handle null cells input and interpret string', async () => {
    importerStub.interpret.returns({ id: 'test-id', code: 'describe("sample", () => it("passes", () => {}));' });
    importerStub.import.returns({});

    const failures = await testCells(null);
    assert.strictEqual(failures, 0);
    sinon.assert.called(importerStub.interpret);
    sinon.assert.called(importerStub.import);
  });

  it('should interpret a string path (non-ipynb)', async () => {
    importerStub.interpret.returns([{ id: 'test-id', code: 'describe("test", function () {})' }]);
    importerStub.import.returns({});

    const failures = await testCells('test.js');
    assert.strictEqual(failures, 0);
  });

  it('should wrap raw code string as an array if not array', async () => {
    importerStub.interpret.returns({ id: 'inline-code', code: 'describe("inline", function () {})' });
    importerStub.import.returns({});

    const failures = await testCells('describe("inline", function () {})');
    assert.strictEqual(failures, 0);
  });

  it('should interpret array of cell objects', async () => {
    const cells = [{ id: 'a', code: 'describe("suite", function () {})' }];
    importerStub.import.returns({});

    const failures = await testCells(cells);
    assert.strictEqual(failures, 0);
  });

  it('should handle import error gracefully', async () => {
    const cells = [{ id: 'b', code: 'describe("suite", function () {})' }];
    importerStub.import.throws(new Error('import error'));

    const failures = await testCells(cells);
    assert.strictEqual(failures, 0);
  });
});


## test runner?

watch files run tests?

How to use: 

```
node -e "require('./Core').import('watch files run tests')('**/zuora*.ipynb', 'zuora to eloqua.ipynb')"

node -e "require('./Core').import('watch files run tests')('**/*.ipynb', 'test test runner')"```

TODO: restart this test script every loop? fork and abandon current thread?

TODO: update cell cache in intrepret notebooks.ipynb

TODO: git apply without whitespace, reset the rest?  separate index?  how do make git-scenario app?

TODO: re-import cells on Utilities/.modules folder change. run all "watcher" commands from a test below?  mocha.grep?





In [ ]:
var chokidar = require("chokidar");
var importer = require('../Core');
var testCells = importer.import('test cells');

// TODO: code analysis to combine blocks into modules?

var rateLimiter, done = true;
function testWatcher(files, tests) {
    files = typeof files === 'string' ? [files] : files;
    console.log('watching ' + files + ' - ' + path.resolve('.'))
    var watcher = chokidar.watch(files, {
        interval: 1000,
        atomic: 1000,
        awaitWriteFinish: true
    });
    watcher.on("change", function(event, path) {
        if(!done) {
            return;
        }
        console.log('running all tests');
        done = false;
        return testCells(tests).then(() => (done = true))
    });
    testCells(tests);
    
    var stdin = process.openStdin();
    stdin.addListener("data", function(d) {
        stdin.close();
    });
}
module.exports = testWatcher;



test sigma index?

ROUTE = /test

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Service Graph - Sigma.js</title>
  <style>
    html, body {
      margin: 0;
      height: 100%;
      overflow: hidden;
      font-family: sans-serif;
    }

    #sigma-container {
      width: 100vw;
      height: 100vh;
    }

    #info {
      position: absolute;
      top: 10px;
      left: 10px;
      background: rgba(255, 255, 255, 0.9);
      padding: 10px;
      border-radius: 6px;
      box-shadow: 0 0 5px rgba(0,0,0,0.2);
      z-index: 100;
    }
  </style>
  <!-- Load Sigma.js v2 -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/graphology/0.26.0/graphology.umd.min.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/sigma@2.3.0/build/sigma.min.js"></script>
</head>
<body>

<div id="sigma-container"></div>
<div id="info">Zoom & drag to explore</div>

<script>
  const graph = new graphology.Graph();

  // Example nodes (services)
  const services = [
    'Auth Service', 'User DB', 'Frontend', 'Billing Service',
    'Notification', 'Search Engine', 'Cache', 'API Gateway',
    'Logging', 'Monitoring', 'AI Recommender', 'Backup Service'
  ];

  // Add nodes
  services.forEach((name, i) => {
    graph.addNode(name, {
      label: name,
      x: Math.cos((2 * Math.PI * i) / services.length),
      y: Math.sin((2 * Math.PI * i) / services.length),
      size: 10 + Math.random() * 5,
      color: '#007acc'
    });
  });

  // Add random connections between services
  for (let i = 0; i < services.length * 1.5; i++) {
    const source = services[Math.floor(Math.random() * services.length)];
    const target = services[Math.floor(Math.random() * services.length)];
    if (source !== target && !graph.hasEdge(source, target)) {
      graph.addEdge(source, target, {
        color: '#ccc'
      });
    }
  }

  // Initialize Sigma renderer
  const container = document.getElementById('sigma-container');
  const renderer = new Sigma(graph, container, {
    renderEdgeLabels: false,
    allowInvalidContainer: false,
  });

  // Optional: zoom reset on double-click
  container.addEventListener('dblclick', () => {
    renderer.getCamera().animatedReset();
  });

  // Optional: click node highlight
  renderer.on('clickNode', ({ node }) => {
    alert(`Clicked on: ${node}`);
  });
</script>
</body>
</html>
